In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [23]:
# specify the details of your subscription
SUBSCRIPTION_ID = "e5615bfe-b43b-41ce-bccb-b78867c2ce63"
RESOURCE_GROUP = "rg-dp100-demo-001"
WORKSPACE = 'mlw-dp100-demo'

# get a handle to the subscription and pass the authentication details
load_dotenv("python.env")
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=SUBSCRIPTION_ID, 
                     resource_group_name=RESOURCE_GROUP,
                     workspace_name=WORKSPACE) # note that the kwarg for workspace is not clear, so use positional

In [24]:
from azure.ai.ml.entities import AmlCompute

cluster_basic = AmlCompute(
    name="vmcluster-ml-dev-002",
    type="amlcompute",
    size="STANDARD_D2_v3",
    location="japaneast",
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120
)
ml_client.begin_create_or_update(cluster_basic)

In [34]:
# we can list the compute clusters created:
# note that as compute targets belong to a specific workspace,
# we need to init the MLClient with the workspace name
# else the ValueError: No value for given attribute will be thrown
print([(compute.name, compute.provisioning_state) for compute in ml_client.compute.list()])

# we can get a specific compute cluster by name:
cluster = ml_client.compute.get("vmcluster-ml-dev-002")
cluster.provisioning_state

[('vm-ml-dev', 'Succeeded'), ('vmcluster-ml-dev', 'Succeeded'), ('vmcluster-ml-dev-002', 'Succeeded')]


'Succeeded'

In [32]:
# use try/except to create the compute cluster if it doesn't exist
from azure.core.exceptions import ResourceNotFoundError
# import sleep to wait for the cluster to be created
from time import sleep

cluster_name = "vmcluster-ml-dev-003"
cluster_config = AmlCompute(
    name=cluster_name,
    type="amlcompute",
    size="STANDARD_D2_v3",
    location="japaneast",
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120
)
try:
    cluster = ml_client.compute.get(cluster_name)
    print("Cluster already exists, using it.")
except ResourceNotFoundError:
    print("Cluster not found, creating it.")
    ml_client.begin_create_or_update(cluster_config)


Cluster not found, creating it.
